# 使用pytorch实现CNN
在这一份文档中，我使用了pytorch实现了简单的卷积神经网络，并且使用MNIST数字图像数据集对该神经网络进行训练，最后检验了该神经网络的精确度。
实现可分为以下几个部分。
1. 准备数据
2. 搭建神经网络
3. 训练
4. 测试

## 1. 准备数据
我们可以使用pytorch中自带的函数导入数据。代码如下

In [91]:
import torchvision
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
BATCH_SIZE = 64
TEST_BATCH_SIZE = 1000
LOG_INTERVEL = 10
LEARNING_RATE = 0.01
EPOCH_NUM = 1

In [97]:
mnist_train_dataset = datasets.MNIST('data', train=True, download=True,
                               transform=transforms.Compose([
                                   transforms.ToTensor(),
                                    transforms.Normalize((0.1307,), (0.3081,))
                               ])
                    )
mnist_test_dataset = datasets.MNIST('data', train=False, 
                                transform=transforms.Compose
                               ([
                                   transforms.ToTensor(),
                                   transforms.Normalize((0.1307,), (0.3081,))
                                ])
                    )
train_loader = torch.utils.data.DataLoader(
    mnist_train_dataset,
    batch_size=BATCH_SIZE,
    shuffle=True
)
test_loader = torch.utils.data.DataLoader(
    mnist_test_dataset,
    batch_size=TEST_BATCH_SIZE,
    shuffle=True
)

在看了文档之后，我有一个地方还是没有弄明白：就是那一个transform参数的作用。
不过，总之，到了这里，就能够通过以下的方式得到数据。
其中能够迭代出三个变量
1. batch的索引，这个就是方便自己看而已
2. 一个batch的数据，这里的shape是（64,1,28,28），意思是：一个batch有64张图，其中，每张图的通道数是1，宽和高分别是（28,28）
3. 一个batch中的64张图分别对应的label，也就是图片对应的数据。
接下来就会使用这些数据进行训练。

In [95]:
for data, target in train_loader:
    print(data.numpy().shape, target.numpy().shape)

(64, 1, 28, 28) (64,)
(64, 1, 28, 28) (64,)
(64, 1, 28, 28) (64,)
(64, 1, 28, 28) (64,)
(64, 1, 28, 28) (64,)
(64, 1, 28, 28) (64,)
(64, 1, 28, 28) (64,)
(64, 1, 28, 28) (64,)
(64, 1, 28, 28) (64,)
(64, 1, 28, 28) (64,)
(64, 1, 28, 28) (64,)
(64, 1, 28, 28) (64,)
(64, 1, 28, 28) (64,)
(64, 1, 28, 28) (64,)
(64, 1, 28, 28) (64,)
(64, 1, 28, 28) (64,)
(64, 1, 28, 28) (64,)
(64, 1, 28, 28) (64,)
(64, 1, 28, 28) (64,)
(64, 1, 28, 28) (64,)
(64, 1, 28, 28) (64,)
(64, 1, 28, 28) (64,)
(64, 1, 28, 28) (64,)
(64, 1, 28, 28) (64,)
(64, 1, 28, 28) (64,)
(64, 1, 28, 28) (64,)
(64, 1, 28, 28) (64,)
(64, 1, 28, 28) (64,)
(64, 1, 28, 28) (64,)
(64, 1, 28, 28) (64,)
(64, 1, 28, 28) (64,)
(64, 1, 28, 28) (64,)
(64, 1, 28, 28) (64,)
(64, 1, 28, 28) (64,)
(64, 1, 28, 28) (64,)
(64, 1, 28, 28) (64,)
(64, 1, 28, 28) (64,)
(64, 1, 28, 28) (64,)
(64, 1, 28, 28) (64,)
(64, 1, 28, 28) (64,)
(64, 1, 28, 28) (64,)
(64, 1, 28, 28) (64,)
(64, 1, 28, 28) (64,)
(64, 1, 28, 28) (64,)
(64, 1, 28, 28) (64,)
(64, 1, 28

In [98]:
for data, target in test_loader:
    print(data.numpy().shape, target.numpy().shape)

(1000, 1, 28, 28) (1000,)
(1000, 1, 28, 28) (1000,)
(1000, 1, 28, 28) (1000,)
(1000, 1, 28, 28) (1000,)
(1000, 1, 28, 28) (1000,)
(1000, 1, 28, 28) (1000,)
(1000, 1, 28, 28) (1000,)
(1000, 1, 28, 28) (1000,)
(1000, 1, 28, 28) (1000,)
(1000, 1, 28, 28) (1000,)


## 2. 准备模型
这里我们使用的是适用于图像的卷积神经网络。
总体来说，两层卷积层，两层全连接层（中间间杂着一点dropout层）

In [78]:
class CNNNet(nn.Module):
    def __init__(self):
        super(CNNNet, self).__init__()
        self.conv1 = nn.Conv2d(1, 10, kernel_size=5)
        self.conv2 = nn.Conv2d(10, 20, kernel_size=5)
        self.conv2_drop = nn.Dropout2d()
        self.fc1 = nn.Linear(320, 50)
        self.fc2 = nn.Linear(50, 10)

    def forward(self, x):
        x = F.relu(F.max_pool2d(self.conv1(x), 2))
        x = F.relu(F.max_pool2d(self.conv2_drop(self.conv2(x)), 2))
        x = x.view(-1, 320)
        x = F.relu(self.fc1(x))
        x = F.dropout(x, training=self.training)
        x = self.fc2(x)
        return F.log_softmax(x, dim=1)

关于网络中数据的流动，我一开始看得不是很懂，后来通过print一些参数，还有学习和测试了各个层的作用。以下通过数据维度的变化来说明。
```
(64,1,28,28)
|卷积层（1,10）
(64,10,24,24)
|2*2池化
(64,10,12,12)
|卷积层（10,20）
(64,20,8,8)
|2*2池化
(64,20,4,4)
|拉平
(64, 320）
|全连接层（320-50）
(64,50)
|全连接层（50,-10）
(64,10)
```
最后，64个图片每个得到一个维度是10的向量，再使用logsoftmax，就能够得到最终的结果。
最终的结果中，取最大值，即是该神经网络对该图片预测的结果。

## 3. 训练模型
我们先建立一个模型，并且选择对应的优化器。
写一个训练函数，就可以开始训练了。

### 3.1 编写训练函数
注意，由于dropout层在测试和训练中行为不一致。
我们在代码中使用`model.train()`来使模型进入train模式，开启dropout的功能。

In [74]:
def train(model, device, train_loader, loss_fn, optimizer, epoch):
    model.train() # 注意！
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device) 
        # 对应的数据放到CPU或GPU进行计算
        optimizer.zero_grad()
        output = model(data)
        loss = loss_fn(output, target)
        loss.backward()
        optimizer.step()
        if batch_idx % LOG_INTERVEL == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.item()))

### 3.2 选择具体的模型，优化器，误差函数

为了省时间，这里的EPOCH_NUM在前面设置为1了，因此只训练一次。

In [80]:
my_cnn = CNNNet()
optimizer = optim.SGD(my_cnn.parameters(), lr=LEARNING_RATE)
loss_fn = F.nll_loss

for epoch in range(1, EPOCH_NUM+1):
    train(my_cnn, device, train_loader, loss_fn, optimizer, epoch)

Train Epoch: 1 [0/60000 (0%)]	Loss: 2.295720
Train Epoch: 1 [640/60000 (1%)]	Loss: 2.283223
Train Epoch: 1 [1280/60000 (2%)]	Loss: 2.312423
Train Epoch: 1 [1920/60000 (3%)]	Loss: 2.303166
Train Epoch: 1 [2560/60000 (4%)]	Loss: 2.282536
Train Epoch: 1 [3200/60000 (5%)]	Loss: 2.291557
Train Epoch: 1 [3840/60000 (6%)]	Loss: 2.296117
Train Epoch: 1 [4480/60000 (7%)]	Loss: 2.298089
Train Epoch: 1 [5120/60000 (9%)]	Loss: 2.262919
Train Epoch: 1 [5760/60000 (10%)]	Loss: 2.267450
Train Epoch: 1 [6400/60000 (11%)]	Loss: 2.245378
Train Epoch: 1 [7040/60000 (12%)]	Loss: 2.298362
Train Epoch: 1 [7680/60000 (13%)]	Loss: 2.252747
Train Epoch: 1 [8320/60000 (14%)]	Loss: 2.217358
Train Epoch: 1 [8960/60000 (15%)]	Loss: 2.234322
Train Epoch: 1 [9600/60000 (16%)]	Loss: 2.219873
Train Epoch: 1 [10240/60000 (17%)]	Loss: 2.223683
Train Epoch: 1 [10880/60000 (18%)]	Loss: 2.163524
Train Epoch: 1 [11520/60000 (19%)]	Loss: 2.166460
Train Epoch: 1 [12160/60000 (20%)]	Loss: 2.171826
Train Epoch: 1 [12800/60000 (

## 4 测试模型
训练完一次后，我们想知道模型的效果如何？
前面准备的测试集的数据就在这里用啦。
我们可以拿一整个测试集，放到模型中取得结果，观察以下的值：
1. 分类的精确度
2. 模型本身的误差函数的计算
注意到在测试过程中，dropout层不应该起作用，因此在测试前要运行`model.eval()`
具体函数的作用要查一下文档。

In [108]:
def test(model, device, test_loader, loss_fn):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
#             print(output.detach().numpy().shape)
            test_loss += F.nll_loss(output, target, reduction='sum').item() # sum up batch loss
            pred = output.max(1, keepdim=True)[1] # get the index of the max log-probability
#             print(pred.numpy().shape)
#             print(target.numpy().shape)
#             这里的view_as是让target和pred的维度保持一致，target是（1000，）而pred是（1000,1）
#             需要这一个进行调整
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)
    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

In [109]:
for epoch in range(1, EPOCH_NUM+1):
    test(my_cnn, device, test_loader, loss_fn)


Test set: Average loss: 0.3248, Accuracy: 9088/10000 (91%)

